# Importing needed python packages

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import dbdreader
import matplotlib.pyplot as plt
import os
import scipy as sp
from scipy import odr
from sklearn.linear_model import LinearRegression
import gsw
import matplotlib.dates as mdates
import cmocean.cm as cmo
from mpl_toolkits.basemap import Basemap
import pickle
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from datetime import datetime
import zipfile
from google.cloud import secretmanager
import google_crc32c
# import pyglider as pg
from dbdreader.decompress import Decompressor
import xarray as xr
from matplotlib import colormaps 
from matplotlib.gridspec import GridSpec

import esdglider.gcp as gcp
import esdglider.glider as glider
import esdglider.plots as plots
import esdglider.utils as utils


# Defining initial variables and code set up

In [ ]:
deployment = 'calanus-20241019'
project = "ECOSWIM"
mode = 'delayed'
bucket_name = 'amlr-gliders-deployments-dev'

base_path = "/home/sam_woodman_noaa_gov"
deployments_path = f'{base_path}/{bucket_name}'
config_path = f"{base_path}/glider-lab/deployment-configs"

gcp.gcs_mount_bucket(
    "amlr-gliders-deployments-dev", deployments_path, ro=False)
paths = glider.get_path_deployment(
    project, deployment, mode, deployments_path, config_path)

path_to_sci_ts_nc = os.path.join(paths['tsdir'], f"{deployment}-{mode}-sci.nc")
path_to_eng_ts_nc = os.path.join(paths['tsdir'], f"{deployment}-{mode}-eng.nc")
path_to_sci_gr_nc = os.path.join(paths['griddir'], f"{deployment}_grid-{mode}-1m.nc")

# path_bar = os.path.join(base_path, "ETOPO_2022_v1_15s_N45W135_surface.tif")
# path_foo = os.path.join(base_path, "ETOPO_2022_v1_15s_N45W120_surface.tif")
path_bar = os.path.join(base_path, "ETOPO_2022_v1_15s_N45W135_erddap.nc")
# path_foo = os.path.join(base_path, "ETOPO_2022_v1_15s_N45W120_geoid.nc")


# FYI: on Mac and Linux, you can right click and copy the file to copy the complete file path
# # FYI: on Windows, the slash will need another slash in front so python wil include it. E.g., \\User\\Desktop not \User\Desktop.
# path_to_sci_ts_nc = "/Users/cflaim/Documents/gliders/calanus/ECOSWIM_2024_calanus-20241019_data_nc_timeseries_calanus-20241019-sci.nc"
# # path_to_sci_ts_nc = "/Users/cflaim/Documents/gliders/calanus/ECOSWIM_2024_calanus-20241019_data_nc_gridded_calanus-20241019_grid-1m.nc"
# path_to_eng_ts_nc = "/Users/cflaim/Documents/gliders/calanus/ECOSWIM_2024_calanus-20241019_data_nc_timeseries_calanus-20241019-eng.nc"

# path_to_sci_gr_nc = "/Users/cflaim/Documents/gliders/calanus/ECOSWIM_2024_calanus-20241019_data_nc_gridded_calanus-20241019_grid-1m.nc"

In [ ]:

# def return_var(var):
#     return var

# list_sci_vars = ["latitude",
#     "longitude",
#     "depth",
#     "heading",
#     "pitch",
#     "roll",
#     "waypoint_latitude",
#     "waypoint_longitude",
#     "conductivity",
#     "temperature",
#     "pressure",
#     "chlorophyll",
#     "cdom",
#     "backscatter_700",
#     "oxygen_concentration",
#     "depth_ctd",
#     "distance_over_ground",
#     "salinity",
#     "potential_density",
#     "density",
#     "potential_temperature",
#     "profile_index",
#     "profile_direction"]

# adjustments = {"temperature": return_var, 
#     "chlorophyll":np.log10, 
#     "cdom":np.log10, 
#     "backscatter_700":np.log10, 
#     "oxygen_concentration":return_var,
#     "salinity":return_var, 
#     "density":return_var}

# adjustments_labels = {
#     "chlorophyll":"$log_{10}$", 
#     "cdom":"$log_{10}$", 
#     "backscatter_700":"$log_{10}$", 
# }

# units = {"latitude":"deg",
#     "longitude":"deg",
#     "depth":"m",
#     "heading":"deg",
#     "pitch":"rad",
#     "roll":"rad",
#     "waypoint_latitude":"deg",
#     "waypoint_longitude":"deg",
#     "conductivity":"S$\\bullet m^{-1}$",
#     "temperature":"°C",
#     "pressure":"dbar",
#     "chlorophyll":"µg•$L^{-l}$",
#     "cdom":"ppb",
#     "backscatter_700":"$m^{-1}$",
#     "oxygen_concentration":"µmol•$L^{-1}$",
#     "depth_ctd": "m",
#     "distance_over_ground":"km",
#     "salinity":"PSU",
#     "potential_density":"kg $\\bullet m^{-3}$",
#     "density":"kg $\\bullet m^{-3}$",
#     "potential_temperature":"°C",
#     "profile_index":"1",
#     "profile_direction":"1"}

# sci_colors = {"cdom":cmo.solar, "chlorophyll": cmo.algae, "oxygen_concentration":cmo.tempo, "backscatter_700":colormaps['terrain'], 
#                             "temperature":cmo.thermal, "potential_temperature":cmo.thermal, "salinity":cmo.haline, "density":colormaps['cividis'], 
#                             "potential_density":colormaps['cividis']}

depth_profiles_to_make = ["temperature", "chlorophyll", "cdom", "oxygen_concentration", "backscatter_700", "salinity", "density"] #"oxygen_concentration",
eng_plots_to_make = ["heading",
    "pitch",
    "roll",
    "total_num_inflections",
    "commanded_oil_volume",
    "measured_oil_volume",
    "total_amphr",
    "amphr",
    "battery_voltage",
    "vacuum",
    "leak_detect",
    "leak_detect_forward",
    "leak_detect_science",
    "battpos",
    "target_depth",
    "altitude",
    "distance_over_ground",
    "profile_index",
    "profile_direction"]

In [ ]:
# sci_save_path = os.path.join(paths['plotdir'], "science")
# eng_save_path = os.path.join(paths['plotdir'], "engineering")
path_base = "/home/sam_woodman_noaa_gov/deployment_plots"
save_path = os.path.join(path_base, deployment)
# sci_save_path = os.path.join(path_base, deployment, "science")
# eng_save_path = os.path.join(path_base, deployment, "engineering")

# # sci_save_path = f"/Users/cflaim/Documents/deployment_reports/{project}/{deployment}/plots/science/"
# # eng_save_path = f"/Users/cflaim/Documents/deployment_reports/{project}/{deployment}/plots/engineering/"
# sci_dirs_to_check = [
#     "TS", "timeSections", "spatialSections", "spatialGrids" "maps", 
#     "miscPlots", "timeSeries"]
# eng_dirs_to_check = ["timeSeries", "thisVsThat"]


# if not os.path.isdir(sci_save_path):
#     os.makedirs(sci_save_path)

# if not os.path.isdir(eng_save_path):
#     os.makedirs(eng_save_path)

# # os.chdir(sci_save_path)

# existing_dirs = os.listdir(sci_save_path)
# print(existing_dirs)
# for _dir in sci_dirs_to_check:
#     if _dir not in existing_dirs:
#         os.makedirs(os.path.join(sci_save_path,_dir))

# existing_dirs = os.listdir(eng_save_path)
# # os.chdir(eng_save_path)
# for _dir in eng_dirs_to_check:
#     if _dir not in existing_dirs:
#         os.makedirs(os.path.join(eng_save_path, _dir))

# Gridded Data

### Opening and viewing data sets

In [ ]:
sci_ds_g = xr.load_dataset(path_to_sci_gr_nc)
display(sci_ds_g)
deployment = sci_ds_g.deployment_name
glider = deployment.split("-")[0]
project = sci_ds_g.project
date = deployment.split("-")[1]
location = "Humboldt, CA"
purpose = "in a WEA"

fig_cnt = 1

start = pd.to_datetime(sci_ds_g.time.values.min()).strftime("%Y-%m-%d")
end = pd.to_datetime(sci_ds_g.time.values.max()).strftime("%Y-%m-%d")

plt.scatter(sci_ds_g.time, sci_ds_g.profile)

In [ ]:
# def add_log(var):
#     return adjustments[var](sci_ds_g[var])

# def log_label(var):
#     if var in adjustments_labels.keys():
#         return f"{adjustments_labels[var]}({var} [{units[var]}])"
#     else:
#         return f"{var} [{units[var]}]"


## Depth profiles

In [ ]:
# plots.sci_gridded_loop(sci_ds_g, base_path=save_path, show=True)

In [ ]:
# for var in depth_profiles_to_make:
#     if not var in list(sci_ds_g.data_vars):
#         print(f"Variable name {var} not present in sci_ds_g. Skipping plot")
#         continue
#     plots.sci_timesection_plot(sci_ds_g, var, base_path=save_path).show()

    
#     fig, ax = plt.subplots(figsize=(11, 8.5))
#     std = np.nanstd(sci_ds_g[var])
#     mean = np.nanmean(sci_ds_g[var])

# #     caption = f"Figure {fig_cnt}: Colorized {var} [{units[var]}] plotted with time on the x-axis and depth on the y-axis. \
# # {var[0].upper()}{var[1:]} was obsrved to have a minimum of {sci_ds_g[var].min():0.2f} {units[var]}, \
# # maximum of {sci_ds_g[var].max():0.2f} {units[var]}, mean of {mean:0.2f} {units[var]}, and standard \
# # deviation of {std:0.2f}. These data were collected by a Teledyne Slocum g3 glider named {glider} off of {location} \
# # from {pd.to_datetime(sci_ds_g.time.values.min()).strftime("%Y-%m-%d")} to {pd.to_datetime(sci_ds_g.time.values.max()).strftime("%Y-%m-%d")}. \
# # These data are spatially bound by {sci_ds_g.longitude.min():0.3f}°W, {sci_ds_g.longitude.max():0.3f}°W, {sci_ds_g.latitude.min():0.3f}°N, and {sci_ds_g.latitude.max():0.3f}°N."

#     # if "700" in var:
#     #     ax.pcolormesh(sci_ds_g.time, sci_ds_g.density, sci_ds_g[var]*1e10, cmap=sci_colors[var])

#     p1 = ax.pcolormesh(sci_ds_g.time, sci_ds_g.depth, add_log(var), cmap=sci_colors[var])
#     cbar = fig.colorbar(p1).set_label(label=log_label(var), size=14)
#     ax.invert_yaxis()

#     ax.set_title(f"Glider {glider} for {project}\n std={std:0.2f} mean={mean:0.2f}")
#     ax.set_xlabel(f"Time", fontsize=14)
#     ax.set_ylabel(f"Depth [m]", fontsize=14)
#     # t = ax.text(0, -0.18, caption, horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, wrap=True)

#     for label in ax.get_xticklabels(which='major'):
#         label.set(rotation=15, horizontalalignment='center')
#     fig_cnt += 1
#     plt.savefig(f"{sci_save_path}/timeSections/{deployment}_{var}_timesection.png")
#     # plt.show()

### Spatial sections

In [ ]:
# for var in depth_profiles_to_make:
#     if not var in list(sci_ds_g.data_vars):
#         print(f"Variable name {var} not present in sci_ds_g. Skipping spatial plot")
#         continue
#     plots.sci_spatialsection_plot(sci_ds_g, var, base_path=save_path).show()

#     fig, axs = plt.subplots(1,2,figsize=(11, 8.5), sharey=True)
#     std = np.nanstd(sci_ds_g[var])
#     mean = np.nanmean(sci_ds_g[var])

# #     caption = f"Figure {fig_cnt}: A.) Colorized {var} [{units[var]}] plotted with longitude on the x-axis and depth on the y-axis. \
# # B.) Colorized {var} [{units[var]}] plotted with latitude on the x-axis and depth on the y-axis. \
# # {var[0].upper()}{var[1:]} was obsrved to have a minimum of {sci_ds_g[var].min():0.2f} {units[var]}, \
# # maximum of {sci_ds_g[var].max():0.2f} {units[var]}, mean of {mean:0.2f} {units[var]}, and standard \
# # deviation of {std:0.2f}. These data were collected by a Teledyne Slocum g3 glider named {glider} off of {location} \
# # from {pd.to_datetime(sci_ds_g.time.values.min()).strftime("%Y-%m-%d")} to {pd.to_datetime(sci_ds_g.time.values.max()).strftime("%Y-%m-%d")}. \
# # These data are spatially bound by {sci_ds_g.longitude.min():0.3f}°W, {sci_ds_g.longitude.max():0.3f}°W, {sci_ds_g.latitude.min():0.3f}°N, and {sci_ds_g.latitude.max():0.3f}°N."
        
#     ### Lon
#     p1 = axs[0].pcolormesh(sci_ds_g.longitude, sci_ds_g.depth, add_log(var), cmap=sci_colors[var])
#     # p1 = axs[0].pcolormesh(sci_ds_g.longitude, sci_ds_g.depth, sci_ds_g[var], cmap=sci_colors[var])

#     # cbar = fig.colorbar(p1).set_label(label=f"{var} [{units[var]}]", size=14)
#     axs[0].invert_yaxis()

#     axs[0].set_xlabel(f"Longitude [Deg]", fontsize=14)
#     axs[0].set_ylabel(f"Depth [m]", fontsize=14)
#     axs[0].text(0.05, 0.95, "A.", fontsize=16, ha='left', fontweight="bold", transform=axs[0].transAxes, color="white", antialiased=True)
    
#     ### Lat
#     p2 = axs[1].pcolormesh(sci_ds_g.latitude, sci_ds_g.depth, add_log(var), cmap=sci_colors[var])
#     # p2 = axs[1].pcolormesh(sci_ds_g.latitude, sci_ds_g.depth, sci_ds_g[var], cmap=sci_colors[var])
#     cbar = fig.colorbar(p2).set_label(label=log_label(var), size=14)
#     # axs[1].invert_yaxis()

#     axs[1].set_xlabel(f"Latitude [Deg]", fontsize=14)
#     axs[1].text(0.05, 0.95, "B.", fontsize=16, ha='left', fontweight="bold", transform=axs[1].transAxes, color="white", antialiased=True)
#     # axs[1].set_ylabel(f"Depth [m]", fontsize=14)

#     fig.suptitle(f"Glider {glider} for {project}\n std={std:0.2f} mean={mean:0.2f}")

#     # t = fig.text(0, -0.18, caption, horizontalalignment='left', verticalalignment='center', transform=axs[0].transAxes, wrap=True)

#     for label in axs[0].get_xticklabels(which='major'):
#         label.set(rotation=15, horizontalalignment='center')
#     fig_cnt += 1
#     plt.savefig(f"{sci_save_path}/spatialSections/{deployment}_{var}_spatialSections.png")

#     # plt.show()

In [ ]:
# sci_ds = xr.open_dataset(path_to_sci_ts_nc)
# gs = GridSpec(5, 5,left=0.1, right=0.9, bottom=0.1, top=0.9,wspace=0.05, hspace=0.05)

# for var in depth_profiles_to_make:
#     if not var in list(sci_ds_g.data_vars):
#         print(f"Variable name {var} not present in sci_ds_g. Skipping spatial plot")
#         continue
#     plots.sci_spatialgrid_plot(sci_ds_g, var, base_path=save_path).show()

    # fig = plt.figure(figsize=(11, 8.5))

    # ax0 = fig.add_subplot(gs[0:3, 0:3])
    # ax1 = fig.add_subplot(gs[3:5, 0:3])
    # ax2 = fig.add_subplot(gs[0:3, 3:5])
    # # std = np.nanstd(sci_ds_g[var])
    # # mean = np.nanmean(sci_ds_g[var])
    
    # # _,_,var_ = np.meshgrid(sci_ds_g.longitude.values, sci_ds_g.latitude.values, sci_ds_g[var].sel(depth=0, method='nearest'))
    # # ax0.pcolormesh(sci_ds_g.longitude, sci_ds_g.latitude, var_[:,:,0], cmap=sci_colors[var])
    
    # p = ax0.scatter(sci_ds_g.longitude, sci_ds_g.latitude, c=sci_ds_g[var].sel(depth=0, method='nearest'), cmap=sci_colors[var])

    # ax0.set_ylabel("Latitude [Deg]")
    # ax0.set_xticks([])
    # ax0.set_xticklabels([])

    # # ax0.scatter(sci_ds.longitude, sci_ds.latitude, c=sci_ds[var], cmap=sci_colors[var])
    # ax1.pcolormesh(sci_ds_g.longitude, sci_ds_g.depth, add_log(var), cmap=sci_colors[var])
    # ax1.set_ylabel("Depth [m]")
    # ax1.set_xlabel("Longitude [Deg]")


    # ax1.invert_yaxis()

    # ax2.pcolormesh(sci_ds_g.depth, sci_ds_g.latitude, np.transpose(add_log(var).values), cmap=sci_colors[var])
    # ax2.set_xlabel("Depth [m]")
    # ax2.set_yticks([])
    # ax2.set_yticklabels([])

    # fig.colorbar(p, location='top', ax=[ax2, ax0])


    # # plt.show()
    

# Time series Data

### Opening and viewing the data sets

In [ ]:
# def add_log(var):
#     return adjustments[var](sci_ds[var])

sci_ds = xr.open_dataset(path_to_sci_ts_nc)
display(sci_ds)

# NOTE: click on the data variable drop down to see all of the science parameters

In [ ]:
eng_ds = xr.open_dataset(path_to_eng_ts_nc)
display(eng_ds)

# for key in eng_ds.keys(): print(f"\"{key}\"")

# Depth profiles

# Specific engineering plots

In [ ]:
# plots_to_make = {"oilVol":{"X":eng_ds["commanded_oil_volume"], 
#                            "Y":[eng_ds["measured_oil_volume"]],
#                            "C":["C0"],
#                            "cb":False},
#                  "diveEnergy":{"X":eng_ds["total_num_inflections"], 
#                                "Y":[eng_ds["amphr"], eng_ds["total_amphr"]],
#                                "C":["C0", "C1"],
#                                "cb":False},
#                  "diveDepth":{"X":eng_ds["target_depth"], 
#                               "Y":[eng_ds["depth"]],
#                               "C":["C0"],
#                               "cb":False},
#                  "inflections":{"X":eng_ds["total_num_inflections"], 
#                                 "Y":[eng_ds["total_amphr"]],
#                                 "C":["C0"],
#                                 "cb":False},
#                  "diveAmpHr":{"X":eng_ds["depth"], 
#                               "Y":[eng_ds["amphr"]],
#                               "C":["C0"],
#                               "cb":False},
#                  "leakDetect":{"X":eng_ds["time"], 
#                                "Y":[eng_ds["leak_detect"].rolling(time=900).mean(), 
#                                     eng_ds["leak_detect_forward"].rolling(time=900).mean(), 
#                                     eng_ds["leak_detect_science"].rolling(time=900).mean()], 
#                                 "C":["C0", "C1", "C2"],
#                                 "cb":False},
#                  "vacuumDepth":{"X":eng_ds["time"], 
#                                 "Y":[eng_ds["vacuum"]], 
#                                 "C":[eng_ds["depth"]],
#                                 "cb":True}
#                  }

In [ ]:
# for key in plots_to_make.keys():
#     fig, ax = plt.subplots(figsize=(8.5,8.5))

#     for i in range(len(plots_to_make[key]["Y"])):
#         if key == "oilVol":
#             plot = ax.scatter(plots_to_make[key]["X"], plots_to_make[key]["Y"][i])
            
#         else:
#             plot = ax.scatter(plots_to_make[key]["X"], plots_to_make[key]["Y"][i], label=plots_to_make[key]["Y"][i].name, c = plots_to_make[key]["C"][i])
#         if plots_to_make[key]["cb"]:
#             fig.colorbar(plot)

#     ax.set_xlabel(plots_to_make[key]["X"].name)
#     ax.set_ylabel(plots_to_make[key]["Y"][0].name)
#     ax.legend()
#     plt.show()




In [ ]:
# plots.eng_tvt_loop(eng_ds, save_path, show=True)

# eng_dict = plots.eng_plots_to_make(eng_ds)
# for key in eng_dict.keys():
#     plots.eng_tvt_plot(eng_ds, eng_dict, key, base_path=save_path).show()

# Time series

In [ ]:
plots.eng_timeseries_loop(eng_ds, save_path, True)

# for var in eng_plots_to_make:
#     plots.eng_timeseries_plot(eng_ds, var, base_path=save_path).show()
    # fig, ax = plt.subplots(figsize=(11,8.5))

    # ax.set_xlabel("Time", fontsize=14)
    # ax.set_ylabel(f"{var}", fontsize=14)
    # # ax.invert_yaxis()
    # ax.set_title(f"Glider {glider} for project {project} on {date[0:4]}-{date[4:6]}-{date[6:]}\noff of {location} {purpose}", fontsize=14)

    # p = ax.scatter(eng_ds.time, eng_ds[var], s=3)
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
    # # fig.colorbar(p, location="right").set_label(log_label(var), size=14)

    # for label in ax.get_xticklabels(which='major'):
    #     label.set(rotation=15, horizontalalignment='center')
    # plt.savefig(f"{eng_save_path}/timeSeries/{deployment}_{var}_timeseries.png")
    # plt.show()

In [ ]:
plots.sci_timeseries_loop(sci_ds, save_path, True)
# for var in depth_profiles_to_make:    
#     if not var in list(sci_ds.data_vars):
#         print(f"Variable name {var} not present in sci_ds. Skipping plot")
#         continue
#     plots.sci_timeseries_plot(sci_ds, var, base_path=save_path).show()

    # fig, ax = plt.subplots(figsize=(11,8.5))

    # ax.set_xlabel("Time", fontsize=14)
    # ax.set_ylabel("Depth [m]", fontsize=14)
    # ax.invert_yaxis()
    # ax.set_title(f"Glider {glider} for project {project} on {date[0:4]}-{date[4:6]}-{date[6:]}\noff of {location} {purpose}", fontsize=14)

    # p = ax.scatter(sci_ds.time, sci_ds.depth, c=add_log(var), cmap=sci_colors[var], s=3)
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
    # fig.colorbar(p, location="right").set_label(log_label(var), size=14)

    # for label in ax.get_xticklabels(which='major'):
    #     label.set(rotation=15, horizontalalignment='center')
    # plt.savefig(f"{sci_save_path}/timeSeries/{deployment}_{var}_timeseries.png")

    # # plt.show()

# TS plots

In [ ]:
# # code adapted from Jacob Partida
# s_lims = (np.floor(np.min(sci_ds.salinity)-0.5),
# np.ceil(np.max(sci_ds.salinity)+0.5))

# t_lims = (np.floor(np.min(sci_ds.potential_temperature)-0.5),
#         np.ceil(np.max(sci_ds.potential_temperature)+0.5))
# # print(t_lims)
# S = np.arange(s_lims[0],s_lims[1]+0.1,0.1)
# T = np.arange(t_lims[0],t_lims[1]+0.1,0.1)
# Tg, Sg = np.meshgrid(T,S)
# sigma = gsw.sigma0(Sg,Tg)

# Sg, Tg, sigma = utils.ts_calculations(sci_ds)

# for var in depth_profiles_to_make:
#     if not var in list(sci_ds.data_vars):
#         print(f"Variable name {var} not present in sci_ds. Skipping plot")
#         continue
#     plots.ts_plot(sci_ds, var, base_path=save_path).show()
plots.sci_ts_loop(sci_ds, save_path, True)

#     fig, ax = plt.subplots(figsize=(9.5, 8.5))

#     C0 = ax.contour(Sg, Tg, sigma, colors='grey', zorder=1)
#     C0l = plt.clabel(C0, colors='k', fontsize=9)
#     p0 = ax.scatter(sci_ds.salinity, sci_ds.potential_temperature, c=add_log(var), cmap=sci_colors[var],s=5)
#     cbar0 = fig.colorbar(p0, orientation="vertical", location='right', shrink=1).set_label(label=log_label(var), size=14)

#     ax.set_title(f"{glider} from {start} to {end}", fontsize=14)
#     ax.set_xlabel("Salinity [PSU]", fontsize=14)
#     ax.set_ylabel("Potential temperature [°C]", fontsize=14)
    
#     # plt.savefig(f"{sci_save_path}/TS/{deployment}_{var}_tsPlot.png")
#     plt.show()

# Maps

Load and sanity check the provided ETOPO bathymetry file

In [ ]:
# bar = xr.open_dataset("/Users/cflaim/Downloads/ETOPO_2022_v1_15s_N45W135_surface.nc")
# foo = xr.open_dataset("/Users/cflaim/Downloads/ETOPO_2022_v1_15s_N45W120_surface.nc")
bar = xr.load_dataset(path_bar).rename({'latitude': 'lat', 'longitude': 'lon'})
display(bar)
# foo = xr.open_dataset(path_foo)

# display(bar)
# display(foo)

# foobar = xr.merge([foo, bar])
# display(foobar)

lat, lon = np.meshgrid(bar.lat, bar.lon)

c = plt.pcolormesh(bar.z.lon,bar.z.lat, bar.z, shading='auto')
# c = plt.imshow(foobar.z)
plt.colorbar(c)

In [ ]:
# bar = xr.open_dataset("/Users/cflaim/Downloads/ETOPO_2022_v1_15s_N45W135_surface.nc")
bar = xr.load_dataset(path_bar).rename({'latitude': 'lat', 'longitude': 'lon'})
bar = bar.where(bar.z <= 0, drop=True)
# bar
# np.meshgrid(bar.z.lon, bar.z.lat)


In [ ]:
map_lon_border = 0.1
map_lat_border = 0.2
glider_lon_min = sci_ds_g.longitude.min()
glider_lon_max = sci_ds_g.longitude.max()
glider_lat_min = sci_ds_g.latitude.min()
glider_lat_max = sci_ds_g.latitude.max()

plots.sci_surface_map_loop(sci_ds_g, bar, save_path, True)

# for var in depth_profiles_to_make:    
#     if not var in list(sci_ds_g.data_vars):
#         print(f"Variable name {var} not present in sci_ds. Skipping plot")
#         continue

#     plots.sci_surface_map(sci_ds_g, var, bar, base_path=save_path).show()
    # fig, ax = plt.subplots(figsize = (8.5, 11))
    # ax.set_xlabel('\n\n\nLongitude [Deg]', fontsize=14)
    # ax.set_ylabel('Latitude [Deg]\n\n\n', fontsize=14)
    # m = Basemap(llcrnrlon=glider_lon_min-map_lon_border, llcrnrlat=glider_lat_min-map_lat_border,
    #             urcrnrlon=glider_lon_max+3*map_lon_border, urcrnrlat=glider_lat_max+map_lat_border, projection="merc", resolution='f', ax=ax) # create map object
    #                         # with open(f"/opt/slocumRtDataVisTool/mapPickles/{self.glider}_{glider_lon_mean:0.0f}_{glider_lat_mean:0.0f}", "wb") as fd:
    #                         #     pickle.dump(m, fd, protocol=-1)

    # m.drawcoastlines()
    # m.drawcountries()
    # m.fillcontinents('#e0b479')
    # m.drawlsmask(ocean_color = "#7bcbe3", resolution='f')
    # m.drawparallels(np.linspace(glider_lat_min-map_lat_border, glider_lat_max+map_lat_border, 5), labels=[1,0,0,1], fmt="%0.2f")
    # m.drawmeridians(np.linspace(glider_lon_min-map_lon_border, glider_lon_max+map_lon_border, 5), labels=[1,0,0,1], fmt="%0.3f", rotation=20)
    # m.drawmapscale(glider_lon_max+map_lon_border*1.5, glider_lat_min-map_lat_border/1.5, glider_lon_max-map_lon_border, glider_lat_min+map_lat_border, length=25,barstyle='fancy')

    # x, y = m(sci_ds_g.longitude.values, sci_ds_g.latitude.values)
    # p = m.scatter(x, y, c=sci_ds_g[var].where(sci_ds_g.depth<=10, drop=True).mean(dim="depth"), cmap=sci_colors[var], s=10, zorder=2.5)
    # lon, lat = np.meshgrid(bar.z.lon, bar.z.lat)
    # lon, lat = m(lon, lat)
    # C0 = m.contour(lon, lat, bar.z, levels=4, colors='grey')
    # C0l = plt.clabel(C0, colors='grey', fontsize=9)
    # # m.contourf(lon, lat, bar.z, cmap="Pastel1")


    # fig.colorbar(p, ax=ax, shrink=0.6,location="right").set_label(label=log_label(var), size=14)
    # ax.set_title(f"{glider} 0 - 10m avgerage {var}\nfrom {start} to {end}", fontsize=14)
    # plt.savefig(f"{sci_save_path}/maps/{deployment}_{var}_map_0-10.png")
    # plt.show()

In [ ]:
# bar = xr.open_dataset("/Users/cflaim/Downloads/ETOPO_2022_v1_15s_N45W135_surface.nc")
bar = xr.load_dataset(path_bar).rename({'latitude': 'lat', 'longitude': 'lon'})
bar = bar.where(bar.z <= 0, drop=True)
bar = bar.where(bar.lon <= sci_ds.longitude.max()+.1, drop=True)
bar = bar.where(bar.lon >= sci_ds.longitude.min()-.1, drop=True)
bar = bar.where(bar.lat <= sci_ds.latitude.max()+.1, drop=True)
bar = bar.where(bar.lat >= sci_ds.latitude.min()-.1, drop=True)

bar['z'] = bar['z'] *-1

# bar = bar.where(bar.lon <= 124.2, drop=True)
# bar = bar.where(bar.lon >= 125.2, drop=True)
# bar = bar.where(bar.lat >= 40.7, drop=True)
# bar = bar.where(bar.lon <= 41.15, drop=True)

lon, lat = np.meshgrid(bar.z.lon, bar.z.lat)
# print(lon)
# print(bar.z.sel(lon=-120.00208333333333))


In [ ]:
fig = plt.figure(figsize=(11,8.5))
ax = fig.add_subplot(projection='3d')

bottom = ax.plot_surface(lon, lat, bar.z, cmap="bone_r", rstride=5, cstride=5)
# ax.plot_wireframe(bar.z.lon, bar.z.lat, bar.z, rstride=1, cstride=1)
# ax.contourf(lon, lat, bar.z, zdir='z', offset=-6000)
plot = ax.scatter(sci_ds.longitude, sci_ds.latitude, sci_ds.depth, c=sci_ds.temperature, s=3)

xmin, xmax = sci_ds.longitude.min()-.1, sci_ds.longitude.max()+.1
ymin, ymax = sci_ds.latitude.min()-.1, sci_ds.latitude.max()+.1
ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax])

ax.invert_zaxis()
fig.colorbar(plot, location="right", shrink=0.6).set_label(label="Temperature", size=14)
fig.colorbar(bottom, location="left", shrink=0.6).set_label(label="Bottom Depth [m]", size=14)
# ax.contour(lon, lat, bar.z, zdir='y', offset=44)
# ax.contour(lon[0,:], lat[:,0], bar.z.sel(lat=lat[:,0]), zdir='x', offset=-134)

# ax.contourf(lon, lat, bar.z,zdir='x')

# ax.scatter(bar.z.lon, bar.z.lat, bar.z)
# z.scatter(sci_ds.longitude, sci_ds.latitude, sci_ds.temperature)
ax.view_init(45, -235, 0)
plt.show()